In [1]:
import pyspark

from typing import List, Optional
from pyspark.sql import SparkSession

from delta.tables import DeltaTable

from delta import configure_spark_with_delta_pip

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()


24/08/19 09:55:08 WARN Utils: Your hostname, Emilianos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.129 instead (on interface en0)
24/08/19 09:55:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/emilianofrigo/repositories/my-tests/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/emilianofrigo/.ivy2/cache
The jars for the packages stored in: /Users/emilianofrigo/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b7eab561-1948-4178-911a-09f39d00b3f4;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 243ms :: artifacts dl 27ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   

24/08/19 09:55:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
path_to_json = "/Users/emilianofrigo/repositories/amazon-pipelines/tests/functional/bronze/resources/get_flat_file_all_orders_data_by_last_update_general/expected.json"

df = (
    spark
    .read
    .json(path_to_json)
)

In [9]:
df.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- airbyte_ab_id: string (nullable = true)
 |-- amazon_order_id: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- aud_upd_ts: string (nullable = true)
 |-- filename_adjusted: string (nullable = true)
 |-- ingestion_dt: string (nullable = true)
 |-- input_file_name: string (nullable = true)
 |-- item_id: long (nullable = true)
 |-- last_updated_date: string (nullable = true)
 |-- marketplace_id: string (nullable = true)
 |-- p_entity_id: string (nullable = true)
 |-- p_month: string (nullable = true)
 |-- raw_data: struct (nullable = true)
 |    |-- amazon-order-id: string (nullable = true)
 |    |-- asin: string (nullable = true)
 |    |-- currency: string (nullable = true)
 |    |-- fulfillment-channel: string (nullable = true)
 |    |-- gift-wrap-price: string (nullable = true)
 |    |-- gift-wrap-tax: string (nullable = true)
 |    |-- is-business-order: string (nullable = true)
 |    |-- item-price: string (nullable 

In [10]:
df.show(vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:

df.write.format("delta").mode("overwrite").save("delta_output/")

23/12/15 04:43:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 950745 ms exceeds timeout 120000 ms
23/12/15 04:43:22 WARN SparkContext: Killing executors is not supported by current scheduler.


In [11]:
# write with partition
df.write.format("delta").mode("overwrite").partitionBy("p_month").save("delta_output/")

24/08/20 06:59:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1000320 ms exceeds timeout 120000 ms
24/08/20 06:59:44 WARN SparkContext: Killing executors is not supported by current scheduler.
